In [8]:
if (!requireNamespace("BiocManager", quietly = TRUE))
install.packages("BiocManager")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [9]:
BiocManager::install("TxDb.Hsapiens.UCSC.hg38.knownGene")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.17 (BiocManager 1.30.20), R 4.3.0 (2023-04-21)

Installing package(s) 'BiocVersion', 'TxDb.Hsapiens.UCSC.hg38.knownGene'

also installing the dependencies ‘formatR’, ‘lambda.r’, ‘futile.options’, ‘matrixStats’, ‘futile.logger’, ‘snow’, ‘BH’, ‘MatrixGenerics’, ‘S4Arrays’, ‘DelayedArray’, ‘BiocParallel’, ‘Rhtslib’, ‘SummarizedExperiment’, ‘rjson’, ‘filelock’, ‘GenomeInfoDbData’, ‘plogr’, ‘bitops’, ‘zlibbioc’, ‘XML’, ‘Rsamtools’, ‘GenomicAlignments’, ‘restfulr’, ‘BiocFileCache’, ‘png’, ‘BiocGenerics’, ‘S4Vectors’, ‘IRanges’, ‘GenomeInfoDb’, ‘GenomicRanges’, ‘RSQLite’, ‘RCurl’, ‘XVector’, ‘Biostrings’, ‘BiocIO’, ‘rtracklayer’, ‘biomaRt’, ‘Biobase’, ‘KEGGREST’, ‘GenomicFeatures’, ‘AnnotationDbi’


Old packages: 'bit', 'bslib', 'devtools', 'digest', 'gert', 'highr', 'isoband',
  

In [15]:
library(TxDb.Hsapiens.UCSC.hg38.knownGene)



Because of space limitations, TFEA.ChIPs internal database only includes ChIP-seq experiments from cell types in ENCODE's tiers 1, 2, and 2.5. 
To download the full ReMap2022 database, as well as other ready-to-use databases, visit https://github.com/LauraPS1/TFEA.ChIP_downloads



In [ ]:
#metadata preparation
file1 <- list(
  Name = "SOX21.chr16.macs2_peaks.narrowPeak",
  Accession = "GSM1198993",
  Cell = "ReN_VM",
  Cell_Type = "cancer",
  Treatment = "nontargetingsiRNA",
  Antibody = "Santa_Cruz_SOX2",
  TF = "SOX21"
)
file2 <- list(
  Name = "SOX22.chr16.macs2_peaks.narrowPeak",
  Accession = "GSM1198994",
  Cell = "ReN_VM",
  Cell_Type = "cancer",
  Treatment = "nontargetingsiRNA",
  Antibody = "Santa_Cruz_SOX2",
  TF = "SOX22"
)
metadata <- data.frame(
  Name = c(file1$Name, file2$Name),
  Accession = c(file1$Accession, file2$Accession),
  Cell = c(file1$Cell, file2$Cell),
  Cell.Type = c(file1$Cell_Type, file2$Cell_Type),
  Treatment = c(file1$Treatment, file2$Treatment),
  Antibody = c(file1$Antibody, file2$Antibody),
  TF = c(file1$TF, file2$TF)
)
columnNames <- c("Name", "Accession", "Cell", "Cell.Type",
                 "Treatment", "Antibody", "TF")
metadata <- metadata[columnNames]


    


  # specify folder path containing peak data
  folder <- "peak_data"
  
  # specify metadata file path
  metadata <- as.data.frame(metadata)
  
  
  # create array with the names of the ChIP-Seq files
  file.list <- list.files(folder, pattern=".narrowPeak", full.names=TRUE)
  
  # set format to "bed"
  format <- "narrowPeak"
  
  # convert all files to GenomicRanges objects
  gr.list <- lapply(
    file.list,
    function(file, metadata, format) {
      # read in peak data
      peaks <- read.table(file, header = FALSE, stringsAsFactors = FALSE, sep = "\t")
      # set metadata for this file
      metadata <- metadata[metadata$Name == basename(file), ]
      # convert peaks to GRanges object
      peaks.gr <- txt2GR(peaks, format, metadata)
      return(peaks.gr)
    },
    metadata = metadata,
    format = format
  )
 

  txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene
  Genes <- genes( txdb )

  TF.gene.binding.db <- makeChIPGeneDB( Genes, gr.list ) 
  str( TF.gene.binding.db )
  
  set_user_data(metadata, TF.gene.binding.db )